In [ ]:
import json, os
import pandas as pd
import numpy as np
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.visualize import Visualizer, _normalize_world_keypoints
from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
from aquabyte.data_loader import BODY_PARTS, KeypointsDataset, NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from aquabyte.akpd import AKPD
from matplotlib import pyplot as plt
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

pd.set_option('display.max_rows', 500)

<h1> Load base dataset </h1>

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
query = """
    select * from keypoint_annotations
    where keypoints is not null
    and keypoints -> 'leftCrop' is not null
    and keypoints -> 'rightCrop' is not null
    limit 10000;
"""
df = rds_access_utils.extract_from_database(query)

<h1> Create data transforms </h1>

In [ ]:
class NormalizeCentered2D(object):
    
    """
    Transforms the 2D left and right keypoints such that:
        (1) The center of the left image 2D keypoints is located at the center of the left image
            (i.e. 2D translation)
        (2) The left image keypoints are possibly flipped such that the upper-lip x-coordinate 
            is greater than the tail-notch coordinate. This is done to reduce the total number of 
            spatial orientations the network must learn from -> reduces the training size
        (3) The left image keypoints are then rotated such that upper-lip is located on the x-axis.
            As in (2), this is done to reduce the total number of spatial orientations the network 
            must learn from -> reduces the training size
        (4) Rescale all left image keypoints by some random number between 'lo' and 'hi' args
        (5) Apply Gaussian random noise "jitter" to each keypoint to mimic annotation error
        (5) For all transformations above, the right image keypoint coordinates are accordingly
            transformed such that the original disparity values are preserved for all keypoints
            (or adjusted during rescaling event)
    """


    def flip_center_kps(self, left_kps, right_kps):

        x_min_l = min([kp[0] for kp in left_kps.values()])
        x_max_l = max([kp[0] for kp in left_kps.values()])
        x_mid_l = np.mean([x_min_l, x_max_l])

        y_min_l = min([kp[1] for kp in left_kps.values()])
        y_max_l = max([kp[1] for kp in left_kps.values()])
        y_mid_l = np.mean([y_min_l, y_max_l])

        x_min_r = min([kp[0] for kp in right_kps.values()])
        x_max_r = max([kp[0] for kp in right_kps.values()])
        x_mid_r = np.mean([x_min_r, x_max_r])

        y_min_r = min([kp[1] for kp in right_kps.values()])
        y_max_r = max([kp[1] for kp in right_kps.values()])
        y_mid_r = np.mean([y_min_r, y_max_r])

        fc_left_kps, fc_right_kps = {}, {}
        flip_factor = 1 if left_kps['UPPER_LIP'][0] > left_kps['TAIL_NOTCH'][0] else -1
        for bp in BODY_PARTS:
            left_kp, right_kp = left_kps[bp], right_kps[bp]
            if flip_factor > 0:
                fc_left_kp = np.array([left_kp[0] - x_mid_l, left_kp[1] - y_mid_l])
                fc_right_kp = np.array([right_kp[0] - x_mid_l, right_kp[1] - y_mid_l])
            else:
                fc_right_kp = np.array([x_mid_r - left_kp[0], left_kp[1] - y_mid_r])
                fc_left_kp = np.array([x_mid_r - right_kp[0], right_kp[1] - y_mid_r])
            fc_left_kps[bp] = fc_left_kp
            fc_right_kps[bp] = fc_right_kp

        return fc_left_kps, fc_right_kps


    def _rotate_cc(self, p, theta):
        R = np.array([
            [np.cos(theta), -np.sin(theta)],
            [np.sin(theta), np.cos(theta)]
        ])

        rotated_kp = np.dot(R, p)
        return rotated_kp


    def rotate_kps(self, left_kps, right_kps):
        upper_lip_x, upper_lip_y = left_kps['UPPER_LIP']
        theta = np.arctan(upper_lip_y / upper_lip_x)
        r_left_kps, r_right_kps = {}, {}
        for bp in BODY_PARTS:
            rotated_kp = self._rotate_cc(left_kps[bp], -theta)
            r_left_kps[bp] = rotated_kp
            disp = abs(left_kps[bp][0] - right_kps[bp][0])
            r_right_kps[bp] = np.array([rotated_kp[0] - disp, rotated_kp[1]])

        return r_left_kps, r_right_kps


    def translate_kps(self, left_kps, right_kps, factor):
        t_left_kps, t_right_kps = {}, {}
        for bp in BODY_PARTS:
            left_kp, right_kp = left_kps[bp], right_kps[bp]
            t_left_kps[bp] = factor * np.array(left_kps[bp])
            t_right_kps[bp] = factor * np.array(right_kps[bp])

        return t_left_kps, t_right_kps


    def jitter_kps(self, left_kps, right_kps, jitter):
        j_left_kps, j_right_kps = {}, {}
        for bp in BODY_PARTS:
            j_left_kps[bp] = np.array([left_kps[bp][0] + np.random.normal(0, jitter), 
                                       left_kps[bp][1] + np.random.normal(0, jitter)])
            j_right_kps[bp] = np.array([right_kps[bp][0] + np.random.normal(0, jitter), 
                                        right_kps[bp][1] + np.random.normal(0, jitter)])

        return j_left_kps, j_right_kps



    def modify_kps(self, left_kps, right_kps, factor, jitter, cm):
        fc_left_kps, fc_right_kps = self.flip_center_kps(left_kps, right_kps)
#         r_left_kps, r_right_kps = self.rotate_kps(fc_left_kps, fc_right_kps)
        t_left_kps, t_right_kps = self.translate_kps(fc_left_kps, fc_right_kps, factor)
        j_left_kps, j_right_kps  = self.jitter_kps(t_left_kps, t_right_kps, jitter)
        j_left_kps_list, j_right_kps_list = [], []
        for bp in BODY_PARTS:
            l_item = {
                'keypointType': bp,
#                 'xFrame': j_left_kps[bp][0] + cm['pixelCountWidth'] / 2.0,
#                 'yFrame': j_left_kps[bp][1] + cm['pixelCountHeight'] / 2.0
                'xFrame': j_left_kps[bp][0],
                'yFrame': j_left_kps[bp][1]
            }

            r_item = {
                'keypointType': bp,
#                 'xFrame': j_right_kps[bp][0] + cm['pixelCountWidth'] / 2.0,
#                 'yFrame': j_right_kps[bp][1] + cm['pixelCountHeight'] / 2.0
                'xFrame': j_right_kps[bp][0],
                'yFrame': j_right_kps[bp][1]
            }

            j_left_kps_list.append(l_item)
            j_right_kps_list.append(r_item)

        modified_kps = {
            'leftCrop': j_left_kps_list,
            'rightCrop': j_right_kps_list
        }

        return modified_kps

    
    def __init__(self, lo=None, hi=None, jitter=0.0):
        self.lo = lo
        self.hi = hi
        self.jitter = jitter
    

    def __call__(self, sample):
        keypoints, cm, stereo_pair_id, label = \
            sample['keypoints'], sample['cm'], sample.get('stereo_pair_id'), sample.get('label')
        left_keypoints_list = keypoints['leftCrop']
        right_keypoints_list = keypoints['rightCrop']
        left_kps = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in left_keypoints_list}
        right_kps = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in right_keypoints_list}
        
        factor = 1.0 
        if self.lo and self.hi:
            factor = np.random.uniform(low=self.lo, high=self.hi)
            
        jitter = np.random.uniform(high=self.jitter)
        
        modified_kps = self.modify_kps(left_kps, right_kps, factor, jitter, cm)
        left_kp_input = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in modified_kps['leftCrop']}
        right_kp_input = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in modified_kps['rightCrop']}
        kp_input = {}
        for bp in BODY_PARTS:
            kp_input[bp] = [
                left_kp_input[bp][0] / 4096.0, 
                left_kp_input[bp][1] / 4096.0, 
                right_kp_input[bp][0] / 4096.0, 
                right_kp_input[bp][1] / 4096.0
            ]
        print(kp_input)
        
        sample = {
            'kp_input': kp_input,
            'label': label,
            'stereo_pair_id': stereo_pair_id,
            'cm': cm,
            'single_point_inference': sample.get('single_point_inference')
        }
        
        return sample
    

In [ ]:
class ToTensor(object):
    
    def __call__(self, sample):
        kp_input, label, stereo_pair_id = \
            sample['kp_input'], sample.get('label'), sample.get('stereo_pair_id')
        
        x = []
        for bp in BODY_PARTS:
            kp_data = kp_input[bp]
            x.append(kp_data)
        if sample.get('single_point_inference'):
            x = np.array([x])
        else:
            x = np.array([x])
        
        kp_input_tensor = torch.from_numpy(x).float()
        label_tensor = torch.from_numpy(np.array([label])).float() if label else None
        
        tensorized_sample = {
            'kp_input': kp_input_tensor,
            'stereo_pair_id': stereo_pair_id
        }
        return tensorized_sample

<h1> Create "bad" Data Transforms </h1>

In [ ]:
df.camera_metadata.iloc[0]

In [ ]:
class KeypointPerturbation(object):
        
    def __init__(self, p_perturbation, min_magnitude, max_magnitude):
        self.p_perturbation = p_perturbation
        self.min_magnitude = min_magnitude
        self.max_magnitude = max_magnitude
        
    def __call__(self, sample):
        keypoints = sample['keypoints']
        left_keypoints, right_keypoints = keypoints['leftCrop'], keypoints['rightCrop']
        perturbed_left_keypoints = []
        
        # pick body parts to perturb (at least one)
        indices = []
        while len(indices) == 0:
            indices = [x for x in range(len(BODY_PARTS)) if (random.random() < self.p_perturbation)]
            
        # apply perturbation
        perturbed_left_keypoints, perturbed_right_keypoints = [], []
        for idx, _ in enumerate(left_keypoints):
            left_item, right_item = left_keypoints[idx], right_keypoints[idx]
            left_perturbation_x, right_perturbation_x, left_perturbation_y, right_perturbation_y = \
                0.0, 0.0, 0.0, 0.0
            if idx in indices:
                case = np.random.choice([0, 1, 2], 1).item()
                if case == 0:
                    left_perturbation_x = np.random.normal(0, np.random.uniform(low=self.min_magnitude, high=self.max_magnitude))
                    right_perturbation_x = np.random.normal(0, np.random.uniform(low=self.min_magnitude, high=self.max_magnitude))
                    left_perturbation_y = np.random.normal(0, np.random.uniform(low=self.min_magnitude, high=self.max_magnitude))
                    right_perturbation_y = np.random.normal(0, np.random.uniform(low=self.min_magnitude, high=self.max_magnitude))
                elif case == 1:
                    x_magnitude = np.random.uniform(low=self.min_magnitude, high=self.max_magnitude)
                    y_magnitude = np.random.uniform(low=self.min_magnitude, high=self.max_magnitude)
                    left_perturbation_x = np.random.normal(0, x_magnitude)
                    right_perturbation_x = np.random.normal(0, abs(x_magnitude + np.random.normal(0, 20)))
                    left_perturbation_y = np.random.normal(0, y_magnitude)
                    right_perturbation_y = np.random.normal(0, abs(y_magnitude + np.random.normal(0, 20)))
                else:
                    k = list(range(len(BODY_PARTS)))
                    k.remove(idx)
                    random_idx = np.random.choice(k, 1).item()
                    left_perturbation_x = left_keypoints[random_idx]['xFrame'] - left_item['xFrame'] + np.random.normal(0, 20)
                    left_perturbation_y = left_keypoints[random_idx]['yFrame'] - left_item['yFrame'] + np.random.normal(0, 20)
                    right_perturbation_x = right_keypoints[random_idx]['xFrame'] - right_item['xFrame'] + np.random.normal(0, 20)
                    right_perturbation_y = right_keypoints[random_idx]['yFrame'] - right_item['yFrame'] + np.random.normal(0, 20)
                
            perturbed_left_item = {
                'keypointType': left_item['keypointType'],
                'xFrame': left_item['xFrame'] + left_perturbation_x,
                'yFrame': left_item['yFrame'] + left_perturbation_y
            }
            
            perturbed_right_item = {
                'keypointType': right_item['keypointType'],
                'xFrame': right_item['xFrame'] + right_perturbation_x,
                'yFrame': right_item['yFrame'] + right_perturbation_y
            }
            
            perturbed_left_keypoints.append(perturbed_left_item)
            perturbed_right_keypoints.append(perturbed_right_item)
        
        perturbed_keypoints = {
            'leftCrop': perturbed_left_keypoints,
            'rightCrop': perturbed_right_keypoints
        }
        
        transformed_sample = {
            'keypoints': perturbed_keypoints,
            'cm': sample['cm'],
            'stereo_pair_id': sample.get('stereo_pair_id')
        }
        
        return transformed_sample
        

<h1> Establish train / test datasets </h1>

In [ ]:
train_pct = 0.8
train_mask = df.index <= (train_pct * df.shape[0])
val_mask = df.index > (train_pct * df.shape[0])

In [ ]:
good_dataset_train = KeypointsDataset(df[train_mask], transform=transforms.Compose([
                                              NormalizeCentered2D(lo=0.3, hi=2.0, jitter=10),
                                              ToTensor()
                                          ]))

good_dataloader_train = DataLoader(good_dataset_train, batch_size=1, shuffle=True, num_workers=1)

good_dataset_val = KeypointsDataset(df[val_mask], transform=transforms.Compose([
                                              NormalizeCentered2D(lo=0.3, hi=2.0, jitter=10),
                                              ToTensor()
                                          ]))

good_dataloader_val = DataLoader(good_dataset_val, batch_size=1, shuffle=True, num_workers=1)

In [ ]:
bad_dataset_train = KeypointsDataset(df[train_mask], transform=transforms.Compose([
                                              KeypointPerturbation(0.2, 30, 500),
                                              NormalizeCentered2D(lo=0.3, hi=2.0, jitter=10),
                                              ToTensor()
                                          ]))

bad_dataloader_train = DataLoader(bad_dataset_train, batch_size=1, shuffle=True, num_workers=1)

bad_dataset_val = KeypointsDataset(df[val_mask], transform=transforms.Compose([
                                              KeypointPerturbation(0.2, 30, 500),
                                              NormalizeCentered2D(lo=0.3, hi=2.0, jitter=10),
                                              ToTensor()
                                          ]))

bad_dataloader_val = DataLoader(bad_dataset_val, batch_size=1, shuffle=True, num_workers=1)

In [ ]:
class GoodBadKeypointsDataset(Dataset):
    """Good / Bad Keypoints dataset."""

    def __init__(self, good_dataloader, bad_dataloader, transform=None):
        self.good_dataloader = good_dataloader
        self.bad_dataloader = bad_dataloader
        self.process()
        
    def process(self):
        self.X = []
        self.labels = []
        count = 0
        for X_batch in self.good_dataloader:
            if count % 100 == 0:
                print(count)
            count += 1
            x = X_batch['kp_input'].numpy().squeeze()
            self.X.append(x)
            self.labels.append(1)
        for X_batch in self.bad_dataloader:
            if count % 100 == 0:
                print(count)
            count += 1
            x = X_batch['kp_input'].numpy().squeeze()
            self.X.append(x)
            self.labels.append(0)
        

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.labels[idx]

        return x, torch.from_numpy(np.array([y])).float()


In [ ]:
dataset_train = GoodBadKeypointsDataset(good_dataloader_train, bad_dataloader_train)
dataset_val = GoodBadKeypointsDataset(good_dataloader_val, bad_dataloader_val)
dataloader_train = DataLoader(dataset_train, batch_size=25, shuffle=True, num_workers=1)
dataloader_val = DataLoader(dataset_val, batch_size=25, shuffle=True, num_workers=1)

In [ ]:
# TODO: Define your network architecture here
import torch
from torch import nn

class AKPDScorerNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.sigmoid(x)
        return x
        



In [ ]:
network = AKPDScorerNetwork()
epochs = 1000
optimizer = torch.optim.Adam(network.parameters(), lr=0.0001)
criterion = torch.nn.BCELoss()

train_losses, val_losses = [], []

for epoch in range(epochs):
    running_loss = 0.0
    for i, data_batch in enumerate(dataloader_train):
        optimizer.zero_grad()
        X_batch, y_batch = data_batch
        y_pred = network(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 0 and i > 0:
            print(running_loss / i)
    else:
        # print validation loss
        val_loss = 0.0
        with torch.no_grad():
            y_pred_list, y_list = [], []
            for i, data_batch in enumerate(dataloader_val):
                X_batch, y_batch = data_batch
                y_pred = network(X_batch)
                loss = criterion(y_pred, y_batch)
                y_pred_list.extend((y_pred.numpy().squeeze() > 0.5).astype(int).tolist())
                y_list.extend(y_batch.numpy().squeeze().tolist())
                val_loss += loss.item()
                

        val_loss_for_epoch = val_loss / len(dataloader_val)
        y_pred_list, y_list = np.array(y_pred_list), np.array(y_list)
        val_accuracy_for_epoch = sum(y_pred_list == y_list) / len(y_list)
        

    loss_for_epoch = running_loss / len(dataloader_train)
    
    print('-'*20)
    print('Epoch: {}'.format(epoch))
    print('Train Loss: {}'.format(loss_for_epoch))
    print('Validation Loss: {}'.format(val_loss_for_epoch))
    print('Validation Accuracy: {}'.format(val_accuracy_for_epoch))
    
    
    



In [ ]:
torch.save(network, '/root/data/alok/biomass_estimation/playground/akpd_scorer_model.pb')

<h1> Test on Real Examples </h1>

In [ ]:
def display_crops(left_image_f, right_image_f, left_keypoints, right_keypoints, side='both', overlay_keypoints=True, show_labels=False):
    assert side == 'left' or side == 'right' or side == 'both', \
        'Invalid side value: {}'.format(side)

    if side == 'left' or side == 'right':
        fig, ax = plt.subplots(figsize=(20, 10))
        image_f = left_image_f if side == 'left' else right_image_f
        keypoints = left_keypoints if side == 'left' else right_keypoints
        image = plt.imread(image_f)
        ax.imshow(image)

        if overlay_keypoints:
            for bp, kp in keypoints.items():
                ax.scatter([kp[0]], [kp[1]], color='red', s=1)
                if show_labels:
                    ax.annotate(bp, (kp[0], kp[1]), color='red')
    else:
        fig, axes = plt.subplots(2, 1, figsize=(20, 20))
        left_image = plt.imread(left_image_f)
        right_image = plt.imread(right_image_f)
        axes[0].imshow(left_image)
        axes[1].imshow(right_image)
        if overlay_keypoints:
            for bp, kp in left_keypoints.items():
                axes[0].scatter([kp[0]], [kp[1]], color='red', s=1)
                if show_labels:
                    axes[0].annotate(bp, (kp[0], kp[1]), color='red')
            for bp, kp in right_keypoints.items():
                axes[1].scatter([kp[0]], [kp[1]], color='red', s=1)
                if show_labels:
                    axes[1].annotate(bp, (kp[0], kp[1]), color='red')
    plt.show()

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))
# query = """
#     SELECT * FROM prod.crop_annotation ca
#     INNER JOIN prod.annotation_state pas on pas.id=ca.annotation_state_id
#     WHERE ca.service_id = (SELECT ID FROM prod.service where name='LATI')
#     AND ca.left_crop_url is not null
#     AND ca.right_crop_url is not null
#     AND ca.pen_id = 64
#     AND (ca.annotation_state_id=6 OR ca.annotation_state_id=7)
#     AND ca.captured_at > '2019-09-01'
#     LIMIT 40;
# """
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
query = """
        select * from keypoint_annotations 
        where pen_id=61 and captured_at >= '2019-09-13' and captured_at <= '2019-09-20'
        and keypoints -> 'leftCrop' is not null
        and keypoints -> 'rightCrop' is not null
        order by captured_at
        limit 2;
    """
tdf = rds_access_utils.extract_from_database(query)

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
aws_credentials = json.load(open(os.environ['AWS_CREDENTIALS']))
akpd = AKPD(aws_credentials)

keypoints = []
for idx, row in tdf.iterrows():
    left_crop_url, right_crop_url = row.left_crop_url, row.right_crop_url
    left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata
    left_image_f, _, _ = s3_access_utils.download_from_url(left_crop_url)
    right_image_f, _, _ = s3_access_utils.download_from_url(right_crop_url)
    akpd_keypoints = akpd.predict_keypoints(left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata)
    keypoints.append(akpd_keypoints[0])
    



In [ ]:
tdf['keypoints'] = keypoints
tdf.id = tdf.index

In [ ]:
dataset_test = KeypointsDataset(tdf, transform=transforms.Compose([
                                              NormalizeCentered2D(),
                                              ToTensor()
                                          ]))

dataloader_test = DataLoader(dataset_test, batch_size=5, shuffle=False, num_workers=1)


In [ ]:
with torch.no_grad():
    y_pred_list, spid_list = [], []
    for i, data_batch in enumerate(dataloader_test):
        X_batch, spid_batch = data_batch['kp_input'], data_batch['stereo_pair_id']
        y_pred = network(X_batch)
        y_pred_list.extend((y_pred.numpy().squeeze() > 0.5).astype(int).tolist())
        spid_list.extend(spid_batch.numpy().squeeze().tolist())
        






In [ ]:
y_pred_list

In [ ]:
%matplotlib inline
plt.gca().set_aspect('equal', adjustable='box')
X = X_batch[0].numpy().squeeze()
plt.scatter(X[:, 0], X[:, 1])
plt.scatter(X[:, 2], X[:, 3])
plt.grid()
plt.show()

In [ ]:
y_pred_list

In [ ]:
np.where(np.array(y_pred_list) == 1)

In [ ]:
idx = 8
akpd_keypoints = tdf.keypoints.iloc[idx]
left_crop_url, right_crop_url = tdf.left_crop_url.iloc[idx], tdf.right_crop_url.iloc[idx]
left_crop_metadata, right_crop_metadata = tdf.left_crop_metadata.iloc[idx], tdf.right_crop_metadata.iloc[idx]
left_image_f, _, _ = s3_access_utils.download_from_url(left_crop_url)
right_image_f, _, _ = s3_access_utils.download_from_url(right_crop_url)
left_keypoints = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in akpd_keypoints['leftCrop']}
right_keypoints = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in akpd_keypoints['rightCrop']}
display_crops(left_image_f, right_image_f, left_keypoints, right_keypoints)

In [ ]:
for i, X_batch in enumerate(bad_dataloader_train):
    if i == 2:
        print(X_batch)
        break

In [ ]:
%matplotlib inline
plt.gca().set_aspect('equal', adjustable='box')
X = X_batch['kp_input'].numpy().squeeze()
plt.scatter(X[:, 0], X[:, 1])
plt.scatter(X[:, 2], X[:, 3])
plt.grid()
plt.show()